In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/Mia-Github/Colabs/git-pytorch-projects/pytorch-projects

/content/drive/MyDrive/Mia-Github/Colabs/git-pytorch-projects/pytorch-projects


### Step1. Data 분할 위한 디렉토리 생성 - Train / Validation / Test



In [ ]:
import os
import shutil

In [ ]:
#1. 원본 데이터(with 클래스별 하위 디렉토리 포함)가 있는 곳
original_dataset_dir = './dataset'
class_list = os.listdir(original_dataset_dir)

In [ ]:
#2. train, val, test 폴더 포함할 디렉토리 생성
base_dir = './splitted'
os.mkdir(base_dir)

In [ ]:
#3. 하위 train, val, test 폴더 생성
train_dir = os.path.join(base_dir, 'train') #./splitted/train
os.mkdir(train_dir)

val_dir = os.path.join(base_dir, 'val') #./splitted/val
os.mkdir(val_dir)

test_dir = os.path.join(base_dir, 'test') #./splitted/test
os.mkdir(test_dir)

In [ ]:
#3. 하위 train, val, test 폴더 내 클래스별 폴더 생성
for clss in class_list:
  os.mkdir(os.path.join(train_dir, clss))
  os.mkdir(os.path.join(val_dir, clss))
  os.mkdir(os.path.join(test_dir, clss))

### Step2. 데이터 분할과 클래스별 데이터 수 확인

In [ ]:
import math

In [ ]:
for clss in class_list:
  path = os.path.join(original_dataset_dir, clss) #./dataset/class명
  fnames = os.listdir(path) #path 내 파일들

  train_size = math.floor(len(fnames)*0.6)
  val_size = math.floor(len(fnames)*0.2)  
  test_size = math.floor(len(fnames)*0.2)

  train_fnames = fnames[:train_size]
  for f in train_fnames:
    src = os.path.join(path, f) #./dataset/class명/파일명
    dst = os.path.join(os.path.join(train_dir, clss), f) #./splitted/train/class명/파일명
    shutil.copyfile(src, dst)

  val_fnames = fnames[train_size:train_size+val_size]
  for f in val_fnames:
    src = os.path.join(path, f) #./dataset/class명/파일명
    dst = os.path.join(os.path.join(val_dir, clss), f) #./splitted/val/class명/파일명
    shutil.copyfile(src, dst) 

  test_fnames = fnames[train_size+val_size:]
  for f in test_fnames:
    src = os.path.join(path, f) #./dataset/class명/파일명
    dst = os.path.join(os.path.join(test_dir, clss), f) #./splitted/test/class명/파일명
    shutil.copyfile(src, dst) 

### Step3. 데이터셋 준비 완료, Dataloader 생성

In [ ]:
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

In [ ]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device('cuda' if USE_CUDA else 'cpu')

In [ ]:
BATCH_SIZE = 256
EPOCH = 30

In [ ]:
#transforms.Compose()
#이미지 데이터 전처리 / 텐서 형태로 변환
transform_base = transforms.Compose([transforms.Resize((64,64)), transforms.ToTensor()])

In [ ]:
#ImageFolder
#데이터셋을 불러오는 메소드
#하나의 클래스 = 하나의 폴더일 경우 사용
train_dataset = ImageFolder(root='./splitted/train', transform=transform_base)
val_dataset = ImageFolder(root='./splitted/val', transform=transform_base)

In [ ]:
#DataLoader
#불러온 이미지 데이터를 주어진 조건에 따라 미니 배치 단위로 분리
#shuffle을 이용해 데이터의 순서가 섞여 모델이 학슶
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)